In [1]:
!pip install ftfy regex tqdm torch
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch-lightning

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-n8sdqrio
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-n8sdqrio
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620


In [2]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


In [3]:
pl.seed_everything(CONFIG.global_random_state)

Global seed set to 101


101

In [4]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_gen_cap.csv")


In [5]:
col_to_test = [c for c in clean_df.columns.values if "caption" in c ]
# col_to_test = ["caption"]

In [6]:
col_to_test

['caption', 'gen_caption_ft', 'gen_caption_base']

In [7]:
image_size = 384
model_url = '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/Retrieval_aliproduct2_filtered/save_checkpoint_4.pth'

preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
clip_model =ALIPRODUCT_BLIP(model_url,image_size,vit='base')

load checkpoint from /home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/Retrieval_aliproduct2_filtered/save_checkpoint_4.pth


In [8]:
def test_feature(caption_col):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_gen_cap.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,CONFIG.test_batch_size,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    faiss_index = faiss.IndexFlatIP(256)
    print(image_embed.numpy().astype(np.float32).shape)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred",sum(top10_preds))

    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc,text_embed

In [9]:
# def topk_accuracy(y_true, pred, label_map, k,threshold):
#     pred[pred < threshold] = 0
#     pred_count = []
#     remap = np.vectorize(lambda x: label_map[x]["label"])
#     #get top k values
#     top_k_pred = remap(pred.argsort(axis=1)[:, -k:][:, ::-1])
#     # append 1 if true label is in top k values else append 0
#     for i in range(len(y_true)):
#         if y_true[i] in top_k_pred[i]:
#             pred_count.append(1)
#         else:
#             pred_count.append(0)
#     return sum(pred_count)/len(pred_count)

In [10]:
col_name = []
top5_acc_preds = []
top10_acc_preds = []
embedding_dict  = {}
for col in col_to_test:
    top5_acc,top10_acc,text_embed = test_feature(col)
    col_name.append(col)
    top5_acc_preds.append(top5_acc)
    top10_acc_preds.append(top10_acc)
    embedding_dict[col] =text_embed
pred_df = {"caption_gen_method":col_name,"top_5":top5_acc_preds,"top_10":top10_acc_preds}

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:17<00:00,  3.51s/it]
(50000, 256)
caption num correct pred 25291
caption num correct pred 31624


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:11<00:00,  3.49s/it]
(50000, 256)
gen_caption_ft num correct pred 17053
gen_caption_ft num correct pred 23198


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:08<00:00,  3.48s/it]
(50000, 256)
gen_caption_base num correct pred 7224
gen_caption_base num correct pred 9946


In [11]:
pred_df = pd.DataFrame(pred_df)

In [12]:
pred_df

,caption_gen_method,top_5,top_10
0,caption,0.50582,0.63248
1,gen_caption_ft,0.34106,0.46396
2,gen_caption_base,0.14448,0.19892


In [28]:
def txt_to_txt_acc(query,key,df):
    faiss_index = faiss.IndexFlatIP(256)
    faiss_index.add(key.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(query.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(query.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [31]:
col_name_txt = []
top5_acc_preds_txt = []
top10_acc_preds_txt = []
for col in embedding_dict.keys():
    top5_acc,top10_acc = txt_to_txt_acc(embedding_dict[col],embedding_dict["caption"],clean_df)
    col_name_txt.append(col)
    top5_acc_preds_txt.append(top5_acc)
    top10_acc_preds_txt.append(top10_acc)
pred_df_txt = {"caption_gen_method":col_name_txt,"top_5_txt_2_txt":top5_acc_preds_txt,"top_10_txt_2_txt":top10_acc_preds_txt}

In [32]:
pred_df_txt = pd.DataFrame(pred_df_txt)
pred_df_txt

,caption_gen_method,top_5_txt_2_txt,top_10_txt_2_txt
0,caption,1.0000,1.00000
1,gen_caption_ft,0.1780,0.26222
2,gen_caption_base,0.0497,0.07656
